--- Day 12: Passage Pathing ---

With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just a path - the only way to know if you've found the best path is to find all of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:

```
start-A
start-b
A-c
A-b
b-d
A-end
b-end
```
This is a list of how all of the caves are connected. You start in the cave named start, and your destination is the cave named end. An entry like b-d means that cave b is connected to cave d - that is, you can move between them.

So, the above cave system looks roughly like this:

```
    start
    /   \
c--A-----b--d
    \   /
     end
```

Your goal is to find the number of distinct paths that start at start, end at end, and don't visit small caves more than once. There are two types of caves: big caves (written in uppercase, like A) and small caves (written in lowercase, like b). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should visit small caves at most once, and can visit big caves any number of times.

Given these rules, there are 10 paths through this example cave system:

```
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end
```

(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave d is never visited by any path: to do so, cave b would need to be visited twice (once on the way to cave d and a second time when returning from cave d), and since cave b is small, this is not allowed.

Here is a slightly larger example:

```
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
```

The 19 paths through it are as follows:

```
start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end
```

Finally, this even larger example has 226 paths through it:


```
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
```

How many paths through this cave system are there that visit small caves at most once?

In [98]:
inp = '''
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
'''.strip().splitlines()

In [100]:
inp = '''
BC-gt
gt-zf
end-KH
end-BC
so-NL
so-ly
start-BC
NL-zf
end-LK
LK-so
ly-KH
NL-bt
gt-NL
start-zf
so-zf
ly-BC
BC-zf
zf-ly
ly-NL
ly-LK
IA-bt
bt-so
ui-KH
gt-start
KH-so
'''.strip().splitlines()

In [72]:
inp

['BC-gt',
 'gt-zf',
 'end-KH',
 'end-BC',
 'so-NL',
 'so-ly',
 'start-BC',
 'NL-zf',
 'end-LK',
 'LK-so',
 'ly-KH',
 'NL-bt',
 'gt-NL',
 'start-zf',
 'so-zf',
 'ly-BC',
 'BC-zf',
 'zf-ly',
 'ly-NL',
 'ly-LK',
 'IA-bt',
 'bt-so',
 'ui-KH',
 'gt-start',
 'KH-so']

In [78]:
from dataclasses import dataclass, field
from pprint import pprint
from collections import Counter

In [75]:
class Node:
    def __init__(self, name):
        self.name = name
        self.links = set()
        
    def __hash__(self):
        return hash((self.name, tuple(sorted([n.name for n in self.links]))))
        
    def __str__(self):
        return f'"N({self.name}, {sorted([n.name for n in self.links])})"'
    
    def __repr__(self):
        return str(self)

        
def build_graph(data):
    all_nodes = {}
    
    for line in data:
        name_a, name_b = line.split('-')
        node_a = all_nodes.get(name_a, Node(name_a))
        node_b = all_nodes.get(name_b, Node(name_b))
        node_a.links.add(node_b)
        node_b.links.add(node_a)
        all_nodes[name_a] = node_a
        all_nodes[name_b] = node_b

    return all_nodes
    
    
def walk(cur: Node, path: list[str]): #, visited: set[str]):
    res = []

    if cur.name == 'end':
        return [[cur.name]]
    
    for n in sorted(cur.links, key=lambda l: l.name):
        if n.name.islower() and n.name in path:
            continue
            
        for p in walk(n, path + [n.name]):
            res.append([cur.name] + p)
            
    return res

def solve1(data):
    all_nodes = build_graph(data)
    
    start = all_nodes['start']
    
    paths = walk(start, [start.name])
    return len(paths) #, paths # all_nodes
    
    
    
        
solve1(inp)

4775

--- Part Two ---

After reviewing the available paths, you realize you might have time to visit a single small cave twice. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named start and end can only be visited exactly once each: once you leave the start cave, you may not return to it, and once you reach the end cave, the path must end immediately.

Now, the 36 possible paths through the first example above are:

```
start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
```

The slightly larger example above now has 103 paths through it, and the even larger example now has 3509 paths through it.



In [101]:
def walk2(cur: Node, path: list[str]):
    res = []

    if cur.name == 'end':
        return [[cur.name]]
    
    for n in sorted(cur.links, key=lambda l: l.name):
        if n.name.islower() and n.name in path:            
            if n.name == 'start':
                continue
            
            c = Counter(path)
#             print(f'checking {n.name}: {path} -> {c}')
            if [
                k for k, v in c.items() 
                if v > 1 and k.islower() 
            ]:            
                continue
            
        for p in walk2(n, path + [n.name]):
            res.append([cur.name] + p)
            
    return res

def solve2(data):
    all_nodes = build_graph(data)
    
    start = all_nodes['start']
    
    paths = walk2(start, [start.name])
    return len(paths), paths # all_nodes
            
solve2(inp)

(152480,
 [['start', 'BC', 'end'],
  ['start', 'BC', 'gt', 'BC', 'end'],
  ['start', 'BC', 'gt', 'BC', 'gt', 'BC', 'end'],
  ['start', 'BC', 'gt', 'BC', 'gt', 'BC', 'ly', 'BC', 'end'],
  ['start', 'BC', 'gt', 'BC', 'gt', 'BC', 'ly', 'BC', 'zf', 'BC', 'end'],
  ['start',
   'BC',
   'gt',
   'BC',
   'gt',
   'BC',
   'ly',
   'BC',
   'zf',
   'NL',
   'bt',
   'NL',
   'so',
   'KH',
   'end'],
  ['start',
   'BC',
   'gt',
   'BC',
   'gt',
   'BC',
   'ly',
   'BC',
   'zf',
   'NL',
   'bt',
   'NL',
   'so',
   'KH',
   'ui',
   'KH',
   'end'],
  ['start',
   'BC',
   'gt',
   'BC',
   'gt',
   'BC',
   'ly',
   'BC',
   'zf',
   'NL',
   'bt',
   'NL',
   'so',
   'LK',
   'end'],
  ['start',
   'BC',
   'gt',
   'BC',
   'gt',
   'BC',
   'ly',
   'BC',
   'zf',
   'NL',
   'bt',
   'so',
   'KH',
   'end'],
  ['start',
   'BC',
   'gt',
   'BC',
   'gt',
   'BC',
   'ly',
   'BC',
   'zf',
   'NL',
   'bt',
   'so',
   'KH',
   'ui',
   'KH',
   'end'],
  ['start',
   'BC',
  